<a href="https://colab.research.google.com/github/palit-ishan/Data-Pipeline/blob/main/Data_Pipelines.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Boiler Plate Code for Creating Database and Loading Data

In [ ]:
""" Installing Required Packages """

' Installing Required Packages '

In [ ]:
!pip install mysql-connector-python

In [ ]:
""" Install My SQL Server and Starting the Server """
!apt-get -y install mysql-server
!service mysql start

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mysql-server is already the newest version (8.0.35-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
 * Starting MySQL database server mysqld
   ...done.


In [ ]:
# Securing Database Connection
!mysql -e "ALTER USER 'root'@'localhost' IDENTIFIED WITH 'mysql_native_password' BY 'root';FLUSH PRIVILEGES;"

ERROR 1045 (28000): Access denied for user 'root'@'localhost' (using password: NO)


In [ ]:
""" Connect to MySQL """
import mysql.connector

# Create a connection to the MySQL server
conn = mysql.connector.connect(user='root', password='root', host='localhost')

# Create a cursor to interact with the MySQL server
cursor = conn.cursor()

In [ ]:
""" Creating and Designing a Database """
# Create a new database named 'library'
cursor.execute("CREATE DATABASE IF NOT EXISTS library")

# Switch to the 'library' database
cursor.execute("USE library")

# Create the 'books' table
cursor.execute('''
CREATE TABLE IF NOT EXISTS books (
    id INT AUTO_INCREMENT PRIMARY KEY,
    title VARCHAR(255) NOT NULL,
    author VARCHAR(255) NOT NULL,
    year_published INT
)
''')

# Always remember to close the cursor and connection when done
cursor.close()
conn.close()

In [ ]:
# Populating the Database
import mysql.connector

# Connect to the MySQL server and the 'library' database
conn = mysql.connector.connect(user='root', password='root', host='localhost', database='library')
cursor = conn.cursor()

books_data = [
    ("To Kill a Mockingbird", "Harper Lee", 1960),
    ("1984", "George Orwell", 1949),
    ("The Great Gatsby", "F. Scott Fitzgerald", 1925)
]

# Insert data using the cursor
cursor.executemany('''
INSERT INTO books (title, author, year_published) VALUES (%s, %s, %s)
''', books_data)

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

In [ ]:
""" Sample Query of Database """
import mysql.connector

# Connect to the MySQL server and the 'library' database
conn = mysql.connector.connect(user='root', password='root', host='localhost', database='library')
cursor = conn.cursor()

# Execute the SELECT query
cursor.execute("SELECT * FROM books")

# Fetch all the results
records = cursor.fetchall()

# Print the records
for record in records:
    print(record)

# Close the cursor and connection
cursor.close()
conn.close()

(1, 'To Kill a Mockingbird', 'Harper Lee', 1960)
(2, '1984', 'George Orwell', 1949)
(3, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925)
(4, 'To Kill a Mockingbird', 'Harper Lee', 1960)
(5, '1984', 'George Orwell', 1949)
(6, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925)


In [ ]:
import mysql.connector

# Connect to the MySQL server and the 'library' database
conn = mysql.connector.connect(user='root', password='root', host='localhost', database='library')
cursor = conn.cursor()
cursor.execute("SELECT * FROM books WHERE year_published < 1950")
classics = cursor.fetchall()
for classic in classics:
  print(classic)

(2, '1984', 'George Orwell', 1949)
(3, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925)
(5, '1984', 'George Orwell', 1949)
(6, 'The Great Gatsby', 'F. Scott Fitzgerald', 1925)


In [ ]:
# Docking
cursor.close()
conn.close()

##Code to scrape data from OpenWeather API using API Key

In [4]:
import requests
import sqlite3
from datetime import datetime

# API Client
def fetch_weather_data(api_key, city):
    url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'
    response = requests.get(url)
    if response.status_code == 200:
        return response.json()
    else:
        print("Failed to fetch data:", response.status_code)
        return None

# Database Storage
def create_connection(db_file):
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except sqlite3.Error as e:
        print(e)
    return None

def create_table(conn):
    create_table_sql = '''
    CREATE TABLE IF NOT EXISTS weather_data (
        id INTEGER PRIMARY KEY,
        city TEXT NOT NULL,
        temperature REAL,
        humidity REAL,
        timestamp DATETIME
    );
    '''
    try:
        c = conn.cursor()
        c.execute(create_table_sql)
    except sqlite3.Error as e:
        print(e)

def insert_data(conn, data):
    sql = ''' INSERT INTO weather_data(city, temperature, humidity, timestamp)
              VALUES(?,?,?,?) '''
    cur = conn.cursor()
    cur.execute(sql, data)
    conn.commit()
    return cur.lastrowid

if __name__ == "__main__":
    api_key = "4d0031a5ae2f0c28e3f8bed5939fabd0"  # Replace with your sample API key
    city = 'New York'  # Sample city

    # Fetch weather data
    weather_data = fetch_weather_data(api_key, city)

    if weather_data:
        # Extract relevant information
        city_name = weather_data['name']
        temperature = weather_data['main']['temp']
        humidity = weather_data['main']['humidity']
        timestamp = datetime.now()

        # Create SQLite database connection
        database = 'weather_data.db'
        conn = create_connection(database)

        if conn:
            # Create table if not exists
            create_table(conn)

            # Insert data into the database
            data_to_insert = (city_name, temperature, humidity, timestamp)
            insert_data(conn, data_to_insert)

            print("Data successfully inserted into the database.")

            # Close database connection
            conn.close()
        else:
            print("Failed to connect to the database.")
    else:
        print("No weather data fetched. Check your API key or city name.")


Data successfully inserted into the database.


##Code for Querying Weather Data from Database (SQLite)

In [5]:
def query_data(conn, query):
    cur = conn.cursor()
    cur.execute(query)
    rows = cur.fetchall()
    return rows

if __name__ == "__main__":
    database = 'weather_data.db'
    conn = sqlite3.connect(database)

    if conn:
        # Example queries
        # Query 1: Retrieve all data
        query1 = "SELECT * FROM weather_data;"
        result1 = query_data(conn, query1)
        print("All weather data:")
        for row in result1:
            print(row)

        # Query 2: Retrieve data for a specific city
        city_name = 'New York'
        query2 = f"SELECT * FROM weather_data WHERE city='{city_name}';"
        result2 = query_data(conn, query2)
        print(f"Weather data for {city_name}:")
        for row in result2:
            print(row)

        # Query 3: Retrieve data for a specific date range (e.g., last 24 hours)
        query3 = "SELECT * FROM weather_data WHERE timestamp >= datetime('now', '-1 day');"
        result3 = query_data(conn, query3)
        print("Weather data for the last 24 hours:")
        for row in result3:
            print(row)

        conn.close()
    else:
        print("Failed to connect to the database.")


All weather data:
(1, 'Boston', 274.7, 92.0, '2024-01-29 00:46:17.491696')
(2, 'New York', 276.25, 89.0, '2024-01-29 00:47:52.353799')
Weather data for New York:
(2, 'New York', 276.25, 89.0, '2024-01-29 00:47:52.353799')
Weather data for the last 24 hours:
(1, 'Boston', 274.7, 92.0, '2024-01-29 00:46:17.491696')
(2, 'New York', 276.25, 89.0, '2024-01-29 00:47:52.353799')
